# **Tutorial** - (semi)-supervised topic modeling
(last updated 26-04-2021)

In this tutorial, we will be looking at a new feature of BERTopic, namely (semi)-supervised topic modeling! This allows us to steer the dimensionality reduction of the embeddings into a space that closely follows any labels you might already have. 

## Semi-supervised modeling
(semi)-supervised topic modeling is a class of methods that allows the user to perform topic modeling with previously defined labels. This might help nudge the model towards specific topics or classes for which you have labels. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts that each is assigned to one of 20 topics:

In [1]:
import pandas as pd
from bertopic import BERTopic
#from sklearn.datasets import fetch_20newsgroups

#data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
#docs = data["data"]
#targets = data["target"]
#target_names = data["target_names"]
#classes = [data["target_names"][i] for i in data["target"]]

In [2]:
import numpy as np
import io
from google.colab import drive
drive.mount('/content/drive')
unlabeled_data = pd.read_csv('/content/drive/My Drive/BERTopic_Unlabeled.csv')
labeled_data = pd.read_csv('/content/drive/My Drive/BERTopic_Labeled.csv')

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
unlabeled_data

,Unnamed: 0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0.1.1,bookId,title,series,author,rating,description,...,Erotica,History,Dystopia,Poetry,Biography,Manga,Thriller,Graphic Novels,Romance,tokens
0,0,1000,1000,28797,274618.The_Complete_Stories_of_Theodore_Sturge...,"The Complete Stories of Theodore Sturgeon, Vol...",The Complete Stories of Theodore Sturgeon #2,"Theodore Sturgeon, Paul Williams (Editor), Sam...",4.21,The second of a planned 10 volumes that will r...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['second', 'planned', 'volumes', 'reprint', 's..."
1,1,1001,1001,6612,14367051-city-of-bones-city-of-ashes-city-of-g...,City of Bones / City of Ashes / City of Glass ...,The Mortal Instruments #1-5,Cassandra Clare (Goodreads Author),4.58,The first five books in the #1 New York Times ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romance,"['first', 'five', 'books', 'new', 'york', 'tim..."
2,2,1002,1002,18812,35156.Clash_of_the_Titans,Clash of the Titans,NaN,Alan Dean Foster,3.88,"PLAYTHING OF THE GODSHe was Perseus, son of Ze...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['plaything', 'godshe', 'perseus', 'son', 'zeu..."
3,3,1003,1003,4134,45791.The_Ballad_of_the_Sad_Caf_and_Other_Stories,The Ballad of the Sad Café and Other Stories,NaN,Carson McCullers,4.00,A classic work that has charmed generations of...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['classic', 'work', 'charmed', 'generations', ..."
4,4,1004,1004,4052,20603820-the-will,The Will,Magdalene #1,Kristen Ashley (Goodreads Author),4.29,"Early in her life, Josephine Malone learned th...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romance,"['early', 'life', 'josephine', 'malone', 'lear..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28647,28647,29647,29647,38855,26162284-katy,Katy,NaN,Jacqueline Wilson,4.36,"Katy Carr is a lively, daredevil oldest sister...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['katy', 'carr', 'lively', 'daredevil', 'oldes..."
28648,28648,29648,29648,11399,29762106-the-anonymous-man,The Anonymous Man,The Anonymous Man #1,Vincent L. Scarsella (Goodreads Author),3.83,"What if one day you could become anonymous, fr...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['one', 'day', 'could', 'become', 'anonymous',..."
28649,28649,29649,29649,2239,608287.Jesus_Son,Jesus' Son,NaN,Denis Johnson,4.10,"Jesus' Son, the first collection of stories by...",...,NaN,NaN,NaN,NaN,NaN,NaN,Thriller,NaN,NaN,"['jesus', 'son', 'first', 'collection', 'stori..."
28650,28650,29650,29650,47747,293453.Akiko_in_the_Castle_of_Alia_Rellapor,Akiko in the Castle of Alia Rellapor,Akiko Books #4,Mark Crilley,4.06,"Akiko and her crew - Spuckler Boach, Mr. Beeba...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Graphic Novels,NaN,"['akiko', 'crew', 'spuckler', 'boach', 'mr', '..."


In [4]:
def make_string(text):
    text_preprocessed = (" ").join(text)
    return text_preprocessed

In [5]:
from ast import literal_eval
unlabeled_data['tokens'] = unlabeled_data['tokens'].apply(lambda row: literal_eval(row))
unlabeled_data['tokens'] = unlabeled_data['tokens'].apply(lambda x: make_string(x))
labeled_data['tokens'] = labeled_data['tokens'].apply(lambda row: literal_eval(row))
labeled_data['tokens'] = labeled_data['tokens'].apply(lambda x: make_string(x))
labeled_data = labeled_data.replace(np.nan, False)
labeled_data['true count'] = labeled_data[['university', 'relationships','break ups', 'divorce', 'weddings', 'death', 'family', 'friendship']].sum(axis=1)

Each document can be put into one of the following categories:

# **(semi)-Supervised modeling**


## Basic Model
Before we start with semi-supervised modeling, let us first take a look at the output of the basic model.

The topics that were created mostly make sense. There are some clearly defined topics such as "nasa, orbit, spacecraft, moon" but also some topics that seem mostly derived from other topics. We can visualize this by extracting the topic representations per class and see if our unsupervised model closely resembles this. 

**NOTE**: You can **hover** over the bars to see the representation per class!!

The results do seem promising. Topics like "nasa, space, etc" seem to be clearly related to sci.space, but some topics were created that span many categories. For example, we expect the topic "bike, bikes, etc"  to only appear in rec.motorcycles.  

## Semi-supervised
In the example above you might notice that some topics were somewhat smushed together. What we would like to see is a clear separation between those topics. Fortunately, we have to labels and can use them to improve the model. 

Since we are not interested in any other topics, this method is called semi-supervised topic modeling. In practice, this means that we have the labels of some documents but not all. 

For this example let's say we only have the labels of all computer-related categories:

When generating our new labels it is important to mark unknown classes as **-1**. Next, we use those newly constructed labels to again run BERTopic:

In [6]:
docs = unlabeled_data['tokens']
target_names = ['university', 'relationships','break ups', 'divorce', 'weddings', 'death', 'family', 'friendship']
topic_model = BERTopic(verbose=True, calculate_probabilities=True)
topics, probs = topic_model.fit_transform(docs)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/896 [00:00<?, ?it/s]

2022-08-02 00:47:34,462 - BERTopic - Transformed documents to Embeddings
2022-08-02 00:48:26,014 - BERTopic - Reduced dimensionality
2022-08-02 00:49:50,440 - BERTopic - Clustered reduced embeddings


In [7]:
topic_model.save("Bertopic_unsupervised_not_guided_8_1")

In [8]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,17536,-1_life_one_world_new
1,0,948,0_dragon_dragons_magic_king
2,1,805,1_school_best_shes_friends
3,2,737,2_killer_detective_case_murder
4,3,365,3_vampire_vampires_blood_dracula
...,...,...,...
229,228,10,228_tita_isabel_natalia_barcelona
230,229,10,229_laroux_hackers_industries_frolick
231,230,10,230_ice_antarctica_atlantis_glinn
232,231,10,231_ben_hope_countdown_anticipation


In [9]:
topic_model.visualize_topics()

In [10]:
topic_model.visualize_barchart()

In [11]:
def refine_topics(LE, z):
  tops, t_similarity = topic_model.find_topics(LE)
  for s in t_similarity:
    if s < z:
      idx = t_similarity.index(s)
      tops.pop(idx)
      t_similarity.remove(s)
  if -1 in tops: 
    id1 = tops.index(-1)
    t_similarity.pop(id1)
    tops.remove(-1)
  d = {LE: tops}
  return d
  

In [12]:
def LE_tops(z):
  LE_tops_dict = {}
  for life_event in ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship']:
    d = refine_topics(life_event, z)
    LE_tops_dict.update(d)
  return LE_tops_dict

In [13]:
LE_Top_Map = LE_tops(0.1)

In [14]:
LE_Top_Map

{'break ups': [20, 225, 1, 4],
 'death': [12, 2, 30, 52],
 'divorce': [206, 6, 121, 42, 4],
 'family': [55, 42, 52, 4],
 'friendship': [1, 42, 151, 40],
 'relationships': [225, 40, 23, 20],
 'university': [226, 1, 23, 62, 118],
 'weddings': [121, 6, 206, 106, 52]}

In [15]:
test_docs = labeled_data['tokens']

In [16]:
test_docs

0      wall street journal amazon bestselling author ...
1      helping set stage biowares hotly anticipated d...
2      sebastian locke fiftysixyearold patriarch powe...
3      take back oxmoon lost paradise childhood take ...
4      mayflower set sail carried board men women wou...
                             ...                        
995    lee wants tarantula member biggest powerful ga...
996    merry adventures robin hood great renown notti...
997    moving presentday oslo brooklyn sleep tells st...
998    captivating sequel inkheart critically acclaim...
999    santa claus dear old friend thief traitor slan...
Name: tokens, Length: 1000, dtype: object

In [17]:
doc_tops, doc_probs = topic_model.transform(test_docs)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2022-08-02 00:50:35,520 - BERTopic - Reduced dimensionality
2022-08-02 00:50:38,704 - BERTopic - Calculated probabilities with HDBSCAN
2022-08-02 00:50:38,706 - BERTopic - Predicted clusters


In [18]:
len(doc_probs[0])

233

In [19]:
#for each doc, for each LE, get list of probabilities of topic in LE. If any probability in list > 0.5, classify as LE. 
def results_z(doc_probs, z):
  LE_Top_Map = LE_tops(z)
  results = []
  for doc in doc_probs:
    temp = []
    for LE in ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship']:
      LE_topics = LE_Top_Map.get(LE)
      #print(LE_topics)
      pred = False
      for t in LE_topics:
        if doc[t-1] > 0.5:
          pred = True
      temp.append(pred)
    results.append(temp)
  return results

In [20]:
#for z in [0.1, 0.2, 0.3, 0.4, 0.5]:
%%time
results = results_z(doc_probs, 0.5)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_5.csv")
!cp bertopic_unsup_5.csv "drive/My Drive/"

CPU times: user 120 ms, sys: 113 ms, total: 234 ms
Wall time: 289 ms


In [21]:
results = results_z(doc_probs, 0.4)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_4.csv")
!cp bertopic_unsup_4.csv "drive/My Drive/"

In [22]:
results = results_z(doc_probs, 0.3)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_3.csv")
!cp bertopic_unsup_3.csv "drive/My Drive/"

In [23]:
results = results_z(doc_probs, 0.2)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_2.csv")
!cp bertopic_unsup_2.csv "drive/My Drive/"

In [24]:
results = results_z(doc_probs, 0.1)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_1.csv")
!cp bertopic_unsup_1.csv "drive/My Drive/"

In [25]:
results = results_z(doc_probs, 0.01)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_01.csv")
!cp bertopic_unsup_01.csv "drive/My Drive/"

In [26]:
results = results_z(doc_probs, 0.001)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_001.csv")
!cp bertopic_unsup_001.csv "drive/My Drive/"

In [27]:
results = results_z(doc_probs, 0.0001)
results_df = pd.DataFrame(results, columns = ['university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'])
results_df.to_csv("bertopic_unsup_0001.csv")
!cp bertopic_unsup_0001.csv "drive/My Drive/"

In [28]:
#topic to life_event refinement
#remove -1 from topics
#remove all topics < 0.5
#map LE to topics
#if topic probability > 0.5

Finally, we can again extract the topics per class to see if our semi-supervised approach had some effect:

We can clearly see that many more topics about computers were created and that the seperation between those topics are solid. This indicates that even if you do not have all the labels, you can definitely improve the model!

However, there are still some clusters that could be improved with the labels that we have. 

## Supervised

Finally, we are going to be using all labels. These labels help BERTopic understand where most clusters can be found. However, this does not mean that it will only find the 20 clusters that we have defined. If there are sub-clusters to be found, then there is a good chance BERTopic will find them! 

Not only do we see a nice seperation of the topics, there are significantly less outliers which shows that BERTopic has improved in connecting the documents to topics. 

Let's see the results by again visualizing the topic representation per class: